In [2]:
from data import *
from utils.augmentations import SSDAugmentation

In [3]:
from pathlib import Path

In [4]:
root = Path.home()/'Documents'/'DATASETS'/'VOCdevkit'''
cfg = voc

In [38]:
dataset = VOCDetection(root = root,
                      transform=SSDAugmentation(cfg['min_dim'], MEANS))

In [39]:
len(dataset)

16551

In [47]:
sample = dataset[12706]
print(sample[0].size())
print(sample[1].shape)

torch.Size([3, 300, 300])
(5, 5)


In [6]:
import torch.utils.data as data



In [48]:
data_loader = data.DataLoader(dataset, batch_size=4)

batch_iterator = iter(data_loader)

for i, d in enumerate(batch_iterator):
    print(d[1].shape)
    if i ==5:
        break

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 3 and 1 in dimension 1 at /opt/conda/conda-bld/pytorch_1556653215914/work/aten/src/TH/generic/THTensor.cpp:711